In [1]:
# this is week 4 test
!pip install datasets

In [2]:
# Preamble
import sys
sys.path.append('..')

In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

df_train = train_set.to_pandas()
df_val = validation_set.to_pandas()

print(len(df_train))
print(len(df_val))

df_train.head()

116067
13325


,question_text,document_title,language,annotations,document_plaintext,document_url
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...
2,వేప చెట్టు యొక్క శాస్త్రీయ నామం ఏమిటి?,వేప,telugu,"{'answer_start': [12], 'answer_text': ['Azadir...","వేప (లాటిన్ Azadirachta indica, syn. Melia aza...",https://te.wikipedia.org/wiki/%E0%B0%B5%E0%B1%...
3,চেঙ্গিস খান কোন বংশের রাজা ছিলেন ?,চেঙ্গিজ খান,bengali,"{'answer_start': [414], 'answer_text': ['বোরজি...",চেঙ্গিজ খান (মঙ্গোলীয়: Чингис Хаан আ-ধ্ব-ব: ...,https://bn.wikipedia.org/wiki/%E0%A6%9A%E0%A7%...
4,రెయ్యలగడ్ద గ్రామ విస్తీర్ణత ఎంత?,రెయ్యలగడ్ద,telugu,"{'answer_start': [259], 'answer_text': ['27 హె...","రెయ్యలగడ్ద, విశాఖపట్నం జిల్లా, గంగరాజు మాడుగుల...",https://te.wikipedia.org/wiki/%E0%B0%B0%E0%B1%...


In [4]:
# Get train and validation data for each language
df_train_bengali = df_train[df_train['language'] == 'bengali']
df_train_arabic = df_train[df_train['language'] == 'arabic']
df_train_indonesian = df_train[df_train['language'] == 'indonesian']

df_val_bengali = df_val[df_val['language'] == 'bengali']
df_val_arabic = df_val[df_val['language'] == 'arabic']
df_val_indonesian = df_val[df_val['language'] == 'indonesian']


# For testing
df_train_english = df_train[df_train['language'] == 'english']
df_val_english = df_val[df_val['language'] == 'english']

In [5]:
def split_text_with_answer(row):
  start = row["annotations"]["answer_start"][0]

  if start == -1:
    return pd.Series([row["document_plaintext"], "", ""])
  end = row["annotations"]["answer_start"][0] + len(row["annotations"]["answer_text"][0])
  before = row["document_plaintext"][:start]
  after = row["document_plaintext"][end:]
  return pd.Series([before, row["annotations"]["answer_text"][0], after])

In [6]:
df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
df_train_english.head()

<ipython-input-6-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
<ipython-input-6-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before", "answer", "after"]] = df_train_english.apply(split_text_with_answer, axis=1)
<ipython-input-6-4102a8e8fa8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...


In [7]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [9]:
def tokenize_before_after(row, tokenizer):
  before_tokens = tokenizer.tokenize(row["before"])
  answer_tokens = tokenizer.tokenize(row["answer"])
  after_tokens = tokenizer.tokenize(row["after"])
  return pd.Series([before_tokens, answer_tokens, after_tokens])

In [10]:
df_train_english[["before_tokens", "answer_tokens", "after_tokens"]] = df_train_english.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
df_train_english.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (985 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-10-7dda50543443>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english[["before_tokens", "answer_tokens", "after_tokens"]] = df_train_english.apply(lambda row: tokenize_before_after(row, mbert_tokeniser), axis=1)
<ipython-input-10-7dda50543443>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]"
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators..."


In [11]:
max_len = df_train_english.apply(lambda row: len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]), axis=1).max()
print(max_len)

4116


In [12]:
df_train_english["text_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len - (len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]))), axis=1)
df_train_english.head()

<ipython-input-12-087f3c0744ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["text_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len - (len(row["before_tokens"]) + len(row["answer"]) + len(row["after_tokens"]))), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."


In [13]:
def create_BIO_tags_text(row):
  before = ["O"] * len(row["before_tokens"])
  if len(row["answer_tokens"]) == 0:
    answer = []
  else:
    answer = ["B"] + ["I"] * (len(row["answer_tokens"]) - 1)
  after = ["O"] * len(row["after_tokens"])
  pad = ["O"] * len(row["text_pad"])
  return before + answer + after + pad

In [14]:
df_train_english["BIO_tags_text"] = df_train_english.apply(create_BIO_tags_text, axis=1)
df_train_english.head()

<ipython-input-14-d523b1fb91be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["BIO_tags_text"] = df_train_english.apply(create_BIO_tags_text, axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ..."


In [16]:
df_lengts_not_eq = df_train_english[df_train_english.apply(lambda row: not (len(row["before_tokens"]) + len(row["answer_tokens"]) + len(row["after_tokens"]) + len(row["text_pad"]) == len(row["BIO_tags_text"])), axis=1)]
df_lengts_not_eq

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text


In [17]:
# tokenize question:
tokenizer = mbert_tokeniser
df_train_english["question_tokens"] = df_train_english.apply(lambda row: tokenizer.tokenize(row["question_text"]), axis=1)
df_train_english.head()

<ipython-input-17-f1d6a7642d2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["question_tokens"] = df_train_english.apply(lambda row: tokenizer.tokenize(row["question_text"]), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]"
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]"
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]"
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]"


In [18]:
max_len_questions = df_train_english.apply(lambda row: len(row["question_tokens"]), axis=1).max()
print(max_len_questions)

30


In [19]:
df_train_english["question_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len_questions - len(row["question_tokens"])), axis=1)
df_train_english.head()

<ipython-input-19-893fd88b38cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["question_pad"] = df_train_english.apply(lambda row: ["[PAD]"] * (max_len_questions - len(row["question_tokens"])), axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA..."


In [20]:
df_train_english["BIO_tags_question"] = df_train_english.apply(lambda row: ["O"] * max_len_questions, axis=1)
df_train_english.head()

<ipython-input-20-585ae4f82ce9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["BIO_tags_question"] = df_train_english.apply(lambda row: ["O"] * max_len_questions, axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [21]:
df_train_english["merged_BIO"] = df_train_english.apply(lambda row: row["BIO_tags_text"] + row["BIO_tags_question"], axis=1)
df_train_english.head()

<ipython-input-21-2b7725de0782>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["merged_BIO"] = df_train_english.apply(lambda row: row["BIO_tags_text"] + row["BIO_tags_question"], axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ...","[what, do, grasshoppers, eat, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[B, I, I, I, I, I, I, I, I, I, I, I, I, I, I, ..."


In [30]:
df_train_english["merged_tokens"] = df_train_english.apply(lambda row: row["before_tokens"] + row["answer_tokens"] + row["after_tokens"] + row["text_pad"] + row["question_tokens"] + row["question_pad"], axis=1)
df_train_english.head()

<ipython-input-30-dba44c17613e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_english["merged_tokens"] = df_train_english.apply(lambda row: row["before_tokens"] + row["answer_tokens"] + row["after_tokens"] + row["text_pad"] + row["question_tokens"] + row["question_pad"], axis=1)


,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens
26,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': [159], 'answer_text': ['1920s']}",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,Quantum field theory naturally began with the ...,1920s,.[8]:1,"[quantum, field, theory, naturally, began, wit...",[1920s],"[., [, 8, ], :, 1]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, was, quantum, field, theory, developed, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[quantum, field, theory, naturally, began, wit..."
43,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': [610], 'answer_text': ['Sully...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,The Nobel Prize in Literature (Swedish: Nobelp...,Sully Prudhomme,of France.[3] Each recipient receives a medal...,"[the, nobel, prize, in, literature, (, swedish...","[sull, ##y, pr, ##ud, ##hom, ##me]","[of, france, ., [, 3, ], each, recipient, rece...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, was, the, first, nobel, prize, winner, f...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[the, nobel, prize, in, literature, (, swedish..."
112,When is the dialectical method used?,Dialectic,english,"{'answer_start': [129], 'answer_text': ['disco...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"Dialectic or dialectics (Greek: διαλεκτική, di...",discourse between two or more people holding d...,". Dialectic resembles debate, but the concept ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ...","[discourse, between, two, or, more, people, ho...","[., dialect, ##ic, res, ##em, ##bles, debate, ...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[when, is, the, dialect, ##ical, method, used, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[dialect, ##ic, or, dialect, ##ics, (, greek, ..."
123,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': [88], 'answer_text': ['Sejong...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,Hangul was personally created and promulgated ...,Sejong the Great,".[1][2] Sejong's scholarly institute, the Hall...","[hangul, was, personally, created, and, promu,...","[se, ##jong, the, great]","[., [, 1, ], [, 2, ], se, ##jong, ', s, schola...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[who, invented, hangul, ?]","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[hangul, was, personally, created, and, promu,..."
125,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': [0], 'answer_text': ['Grassho...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,,"Grasshoppers are plant-eaters, with a few spec...",. They protect themselves from predators by ca...,[],"[grasshoppers, are, plant, -, eat, ##ers, ,, w...","[., they, protect, themselves, from, predators...","[[PAD], [PAD], [PAD], [PAD], [PAD], [PAD], [PA...","[B, I, I, I, I, I, I, I, I, 

In [32]:
df_lengts_not_eq = df_train_english[df_train_english.apply(lambda row: not (len(row["merged_BIO"])) == len(row["merged_tokens"]), axis=1)]
df_lengts_not_eq

,question_text,document_title,language,annotations,document_plaintext,document_url,before,answer,after,before_tokens,answer_tokens,after_tokens,text_pad,BIO_tags_text,question_tokens,question_pad,BIO_tags_question,merged_BIO,merged_tokens


# Format of merged vector:
[before] + [answer] + [after] + [padding] + [question] + [padding]